<a href="https://colab.research.google.com/github/taariq20/816042980COMP3610Assignment1/blob/main/816042980COMP3610A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Part 1: Data Ingestion And Storage

In [ ]:
import os
import requests
import pandas as pd

# Create raw data directory
raw_path = "data/raw"

if not os.path.exists(raw_path):
    os.makedirs(raw_path)

trip_url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
zone_url = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"

trip_file = os.path.join(raw_path, "yellow_tripdata_2024-01.parquet")
zone_file = os.path.join(raw_path, "taxi_zone_lookup.csv")


In [ ]:
def download_file(url, save_path):
    if os.path.exists(save_path):
        print(f"{save_path} already exists.")
        return

    print(f"Downloading {save_path}...")
    response = requests.get(url)

    if response.status_code == 200:
        with open(save_path, "wb") as f:
            f.write(response.content)
        print("Download complete.\n")
    else:
        raise Exception(f"Failed to download {url}")

download_file(trip_url, trip_file)
download_file(zone_url, zone_file)

print("All files downloaded successfully.")


data/raw/yellow_tripdata_2024-01.parquet already exists.
data/raw/taxi_zone_lookup.csv already exists.
All files downloaded successfully.


In [ ]:
print("Loading dataset...")
df = pd.read_parquet(trip_file)

print("Dataset loaded successfully.\n")

expected_columns = [
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "PULocationID",
    "DOLocationID",
    "passenger_count",
    "trip_distance",
    "fare_amount",
    "tip_amount",
    "total_amount",
    "payment_type"
]

print("Validating required columns...")
for col in expected_columns:
    assert col in df.columns, f"Missing required column: {col}"

print("All required columns present.\n")

print("Validating datetime columns...")
assert pd.api.types.is_datetime64_any_dtype(df["tpep_pickup_datetime"]), \
    "Pickup datetime not correct type"

assert pd.api.types.is_datetime64_any_dtype(df["tpep_dropoff_datetime"]), \
    "Dropoff datetime not correct type"

print("Datetime columns validated.\n")

print("Dataset Info:")
print(df.info())

print("\nTotal Rows:", df.shape[0])
print("\nSummary Statistics:")
print(df.describe())

print("\nPart 1 completed successfully.")


Loading dataset...
Dataset loaded successfully.

Validating required columns...
All required columns present.

Validating datetime columns...
Datetime columns validated.

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amo

PART 2: Data Transformation & Analysis

In [ ]:
initial_rows = df.shape[0]

# Remove nulls in critical columns
df_clean = df.dropna(subset=[
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "PULocationID",
    "DOLocationID",
    "fare_amount"
])

after_nulls = df_clean.shape[0]

# Remove invalid trips
df_clean = df_clean[
    (df_clean["trip_distance"] > 0) &
    (df_clean["fare_amount"] > 0) &
    (df_clean["fare_amount"] <= 500)
]

after_invalid = df_clean.shape[0]

# Remove trips with invalid timestamps
df_clean = df_clean[
    df_clean["tpep_dropoff_datetime"] > df_clean["tpep_pickup_datetime"]
]

final_rows = df_clean.shape[0]

print("Rows removed (null values):", initial_rows - after_nulls)
print("Rows removed (invalid values):", after_nulls - after_invalid)
print("Rows removed (invalid timestamps):", after_invalid - final_rows)
print("Final row count:", final_rows)


Rows removed (null values): 0
Rows removed (invalid values): 94940
Rows removed (invalid timestamps): 112
Final row count: 2869572


In [ ]:
# Trip duration in minutes
df_clean["trip_duration_minutes"] = (
    (df_clean["tpep_dropoff_datetime"] - df_clean["tpep_pickup_datetime"])
    .dt.total_seconds() / 60
)

# Trip speed
df_clean["trip_speed_mph"] = (
    df_clean["trip_distance"] /
    (df_clean["trip_duration_minutes"] / 60)
)

# Pickup hour
df_clean["pickup_hour"] = df_clean["tpep_pickup_datetime"].dt.hour

# Pickup day of week
df_clean["pickup_day_of_week"] = df_clean["tpep_pickup_datetime"].dt.day_name()

print("Feature engineering completed.")


Feature engineering completed.


In [ ]:
import duckdb

con = duckdb.connect()

con.register("trips", df_clean)
con.execute(f"CREATE TABLE zones AS SELECT * FROM read_csv_auto('{zone_file}')")


In [ ]:
query1 = """
SELECT z.Zone, COUNT(*) AS trip_count
FROM trips t
JOIN zones z ON t.PULocationID = z.LocationID
GROUP BY z.Zone
ORDER BY trip_count DESC
LIMIT 10;
"""
result1 = con.execute(query1).df()
result1


,Zone,trip_count
0,Midtown Center,140141
1,Upper East Side South,140118
2,JFK Airport,138427
3,Upper East Side North,133962
4,Midtown East,104342
5,Times Sq/Theatre District,102958
6,Penn Station/Madison Sq West,102152
7,Lincoln Square East,101794
8,LaGuardia Airport,87693
9,Upper West Side South,86466


In [ ]:
query2 = """
SELECT pickup_hour, AVG(fare_amount) AS avg_fare
FROM trips
GROUP BY pickup_hour
ORDER BY pickup_hour;
"""
result2 = con.execute(query2).df()
result2


,pickup_hour,avg_fare
0,0,19.681288
1,1,17.735781
2,2,16.629330
3,3,18.536212
4,4,23.451590
5,5,27.500120
6,6,22.027144
7,7,18.753927
8,8,17.826515
9,9,17.947134


In [ ]:
query3 = """
SELECT payment_type,
COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() AS percentage
FROM trips
GROUP BY payment_type;
"""
result3 = con.execute(query3).df()
result3


,payment_type,percentage
0,2,14.730873
1,0,4.014362
2,1,80.093721
3,4,0.793010
4,3,0.368034


In [ ]:
query4 = """
SELECT pickup_day_of_week,
AVG(tip_amount / fare_amount) * 100 AS avg_tip_percent
FROM trips
WHERE payment_type = 1
GROUP BY pickup_day_of_week;
"""
result4 = con.execute(query4).df()
result4


,pickup_day_of_week,avg_tip_percent
0,Tuesday,25.730124
1,Wednesday,25.706625
2,Thursday,29.734458
3,Monday,25.514116
4,Friday,25.595701
5,Saturday,26.293995
6,Sunday,25.101118


In [ ]:
query5 = """
SELECT z1.Zone AS pickup_zone,
       z2.Zone AS dropoff_zone,
       COUNT(*) AS trip_count
FROM trips t
JOIN zones z1 ON t.PULocationID = z1.LocationID
JOIN zones z2 ON t.DOLocationID = z2.LocationID
GROUP BY pickup_zone, dropoff_zone
ORDER BY trip_count DESC
LIMIT 5;
"""
result5 = con.execute(query5).df()
result5


,pickup_zone,dropoff_zone,trip_count
0,Upper East Side South,Upper East Side North,21641
1,Upper East Side North,Upper East Side South,19199
2,Upper East Side North,Upper East Side North,15193
3,Upper East Side South,Upper East Side South,14112
4,Midtown Center,Upper East Side South,10139


Part 3: Dashboard Development

In [20]:
import pandas as pd
import plotly.express as px
import joblib

# Save the cleaned dataframe
joblib.dump(df, "df_clean.joblib")
print("Cleaned dataframe saved as df_clean.joblib")

# Use the cleaned dataframe
df = df_clean

# Feature engineering for dashboard
df["pickup_hour"] = df["tpep_pickup_datetime"].dt.hour
df["pickup_date"] = df["tpep_pickup_datetime"].dt.date
df["day_of_week"] = df["tpep_pickup_datetime"].dt.day_name()

df["trip_duration"] = (
    df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
).dt.total_seconds() / 60

df["revenue"] = df["fare_amount"]

df.head()

Cleaned dataframe saved as df_clean.joblib


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,congestion_surcharge,Airport_fee,trip_duration_minutes,trip_speed_mph,pickup_hour,pickup_day_of_week,pickup_date,day_of_week,trip_duration,revenue
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,...,2.5,0.0,19.800000,5.212121,0,Monday,2024-01-01,Monday,19.800000,17.7
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,...,2.5,0.0,6.600000,16.363636,0,Monday,2024-01-01,Monday,6.600000,10.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,...,2.5,0.0,17.916667,15.739535,0,Monday,2024-01-01,Monday,17.916667,23.3
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,...,2.5,0.0,8.300000,10.120482,0,Monday,2024-01-01,Monday,8.300000,10.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,...,2.5,0.0,6.100000,7.868852,0,Monday,2024-01-01,Monday,6.100000,7.9
